In [1]:
import os
import pandas as pd
import numpy as np

import os
import pandas as pd
import numpy as np
from random import randint
from nltk.corpus import wordnet

import nltk
nltk.download('wordnet')

!pip install textstat
import textstat
import nltk.data
nltk.download('punkt')
import random
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 762.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git
os.chdir('bleurt')
!pip install .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116
Receiving objects: 100% (134/134), 31.28 MiB | 23.78 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Processing /content/bleurt
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.8 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456762 sha256=86e52352d49ec05bdaf319d3642686196157a7bd31a09e3217b5b2499a29b686
  Stored in directory: /tmp/pip-ephem-wheel-cache-vy8tik66/wheels/92/4f/fb/afa555fa27aa9e2c7958df797a62cc4e74f0f459cec9c4fa7c
Successfully built BLEU

In [ ]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip .
!unzip BLEURT-20-D6.zip

--2023-08-01 14:47:06--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.11.128, 173.194.218.128, 108.177.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.11.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171135344 (163M) [application/octet-stream]
Saving to: ‘BLEURT-20-D6.zip’

BLEURT-20-D6.zip    100%[===================>] 163.21M   128MB/s    in 1.3s    

2023-08-01 14:47:07 (128 MB/s) - ‘BLEURT-20-D6.zip’ saved [171135344/171135344]

--2023-08-01 14:47:07--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2023-08-01 14:47:07--
Total wall clock time: 1.5s
Downloaded: 1 files, 163M in 1.3s (128 MB/s)
Archive:  BLEURT-20-D6.zip
   creating: BLEURT-20-D6/
  inflating: BLEURT-20-D6/bert_config.json  
  inflating: BLEURT-20-D6/saved_model.pb  
   creating: BLEURT-20-

In [ ]:
#Optional way to speed up BLEURT
'''
!python -m bleurt.score_files \
  -candidate_file=bleurt/test_data/candidates \
  -reference_file=bleurt/test_data/references \
  -bleurt_batch_size=100 \            # Optimization 1.
  -batch_same_length=True \           # Optimization 2.
  -bleurt_checkpoint=BLEURT-20-D12    # Optimization 3.
'''

In [ ]:
! cd /content/bleurt

In [ ]:
#raw data located here: https://drive.google.com/drive/u/1/folders/1UbAnB7QllTsjT4BDyb29O8i7w0dhXDML
train_df = pd.read_csv('train_df.csv') #need to run a few times... might need to look into the weird values
dev_df = pd.read_csv('dev_set.csv')
test_df = pd.read_csv('test_set.csv')

In [ ]:
#df.drop(df[df['lp'] == 'en-ru'].index, inplace = True)

In [ ]:
train_df.head()

,Unnamed: 0,lp,src,mt,ref,system,annotators,domain,year,score
0,119663,zh-en,宋剑锋--人物画名家小品【正版书籍 闪电发货】,Song Jianfeng--Character Painting Famous Artis...,Song Jianfeng - Small Works of Famous Figure P...,HuaweiTSC,1,ecommerce,2022,-5.0
1,37210,en-de,"Mandatory home quarantine in Austria, despite ...","Obligatorische Hausquarantäne in Österreich, t...","Verpflichtende Hausquarantäne in Österreich, t...",VolcTrans-AT,1,news,2021,-0.0
2,87393,en-de,"Recently, I made the switch from an iPhone 12 ...",Kürzlich habe ich den Wechsel vom iPhone 12 Pr...,Kürzlich habe ich von einem iPhone 12 Pro auf ...,bleurt_bestmbr,1,social,2022,-1.0
3,102953,zh-en,双子座流星雨推荐指数图。,Geminid meteor shower recommendation index chart.,Chart of index of recommendation level of the ...,bleurt_bestmbr,1,news,2022,-0.0
4,119786,zh-en,根据酸度的不同可分为三个级别：,It can be divided into three grades according ...,There are three levels according to acidity:,Online-W,1,ecommerce,2022,-1.1


In [ ]:
unique_values = train_df['lp'].unique()
print(unique_values)

['zh-en' 'en-de']


In [ ]:
#import pandas as pd
#from sklearn.model_selection import train_test_split
#train_df, dev_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
len(train_df)

73613

In [ ]:
len(dev_df)

18404

Load in sample data to see if the finetuning is working

In [ ]:
#rename to match the expected JSON formatting
train_df = train_df.rename(columns={"lp": "lp",
                   "src": "src",
                   "mt": "candidate",
                   "ref": "reference",
                   "system": "system",
                   "annotators": "annotators",
                   "domain": "domain",
                   "year": "year",
                   "score": "score"
                   })
dev_df = dev_df.rename(columns={"lp": "lp",
                   "src": "src",
                   "mt": "candidate",
                   "ref": "reference",
                   "system": "system",
                   "annotators": "annotators",
                   "domain": "domain",
                   "year": "year",
                   "score": "score"
                   })

In [ ]:
train_df.head()

,Unnamed: 0,lp,src,candidate,reference,system,annotators,domain,year,score
0,119663,zh-en,宋剑锋--人物画名家小品【正版书籍 闪电发货】,Song Jianfeng--Character Painting Famous Artis...,Song Jianfeng - Small Works of Famous Figure P...,HuaweiTSC,1,ecommerce,2022,-5.0
1,37210,en-de,"Mandatory home quarantine in Austria, despite ...","Obligatorische Hausquarantäne in Österreich, t...","Verpflichtende Hausquarantäne in Österreich, t...",VolcTrans-AT,1,news,2021,-0.0
2,87393,en-de,"Recently, I made the switch from an iPhone 12 ...",Kürzlich habe ich den Wechsel vom iPhone 12 Pr...,Kürzlich habe ich von einem iPhone 12 Pro auf ...,bleurt_bestmbr,1,social,2022,-1.0
3,102953,zh-en,双子座流星雨推荐指数图。,Geminid meteor shower recommendation index chart.,Chart of index of recommendation level of the ...,bleurt_bestmbr,1,news,2022,-0.0
4,119786,zh-en,根据酸度的不同可分为三个级别：,It can be divided into three grades according ...,There are three levels according to acidity:,Online-W,1,ecommerce,2022,-1.1


In [ ]:
train_df.iloc[0,3]

In [ ]:
dev_df.head()

In [ ]:
dev_df.iloc[0,3]

In [ ]:
#filter to only the desired columns for JSON
train_df = train_df[['candidate','reference','score']]
dev_df = dev_df[['candidate','reference','score']]

In [ ]:
train_df

,candidate,reference,score
0,Song Jianfeng--Character Painting Famous Artis...,Song Jianfeng - Small Works of Famous Figure P...,-5.000000
1,"Obligatorische Hausquarantäne in Österreich, t...","Verpflichtende Hausquarantäne in Österreich, t...",-0.000000
2,Kürzlich habe ich den Wechsel vom iPhone 12 Pr...,Kürzlich habe ich von einem iPhone 12 Pro auf ...,-1.000000
3,Geminid meteor shower recommendation index chart.,Chart of index of recommendation level of the ...,-0.000000
4,It can be divided into three grades according ...,There are three levels according to acidity:,-1.100000
...,...,...,...
73608,"Zou Tao said that if the market price is weak,...","Zou Tao said that if the market price is weak,...",-3.666667
73609,While consumption growth provided strong suppo...,Analysts believed that consumption growth had ...,-2.333333
73610,Vice President and Prime Minister of the Unite...,"Mohammed, Vice president and Prime Minister of...",-13.000000
73611,"Don't look at Facebook now very popular, there...",Although Facebook is very popular now with an ...,-16.000000


In [ ]:
#convert everything to a string to ensure finetuning runs appropriately
train_df[['candidate','reference','score']] = train_df[['candidate','reference','score']].astype(str)
dev_df[['candidate','reference','score']] = dev_df[['candidate','reference','score']].astype(str)



In [ ]:
#This will give us the format we need
import json
from json import loads, dumps

train_df.to_json("x_train.json",
           orient="records",
           lines=True)

#Move to the test data folder

In [ ]:
#This will give us the format we need
import json
from json import loads, dumps

dev_df.to_json("x_dev.json",
           orient="records",
           lines=True)

#Move to the test data folder

In [ ]:
!cd ..

In [ ]:
!pwd

/content/bleurt


In [ ]:
#import fintune
from bleurt import finetune
from bleurt.finetune import *

In [ ]:
! cd /content/bleurt/bleurt

In [ ]:
#Run in order to see a list of all of the fine-tuning parameters

!python bleurt/finetune.py -helpfull

In [ ]:
!pwd

/content/bleurt


In [ ]:
!python -m bleurt.finetune \
  -init_bleurt_checkpoint=bleurt/test_checkpoint \
  -model_dir=my_new_bleurt_checkpoint \
  -train_set=bleurt/test_data/x_train.json \
  -dev_set=bleurt/test_data/x_dev.json \
  -num_train_steps=250

2023-08-01 14:52:35.894909: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:tensorflow:Dynamic seq length requested - disabling Eager Mode.
I0801 14:52:39.205442 139880895348736 finetune.py:111] Dynamic seq length requested - disabling Eager Mode.
INFO:tensorflow:Reading BLEURT parameters
I0801 14:52:39.205794 139880895348736 checkpoint.py:32] Reading BLEURT parameters
INFO:tensorflow:Reading paramter from BLEURT checkpoint: bleurt/test_checkpoint
I0801 14:52:39.205883 139880895348736 checkpoint.py:35] Reading paramter from BLEURT checkpoint: bleurt/test_checkpoint
INFO:tensorflow:Config file found, reading.
I0801 14:52:39.206032 139880895348736 checkpoint.py:91] Config file found, reading.
INFO:tensorflow:Will load checkpoint dbleurt_tiny
I0801 14:52:39.206289 139880895348736 checkpoint.py:95] Will load checkpoint dbleurt_tiny
INFO:tensorflow:Loads full paths and checks that files exists.
I0801 14:52:39.206351 139880895348736 checkp

In [ ]:
#downloading zip file to use for future use

!zip -r bleurt_20_D6_initial_checkpoint_test.zip my_new_bleurt_checkpoint/

	zip warning: name not matched: my_new_bleurt_checkpoint/

zip error: Nothing to do! (try: zip -r bleurt_20_D6_initial_checkpoint_test.zip . -i my_new_bleurt_checkpoint/)


In [2]:
import pandas as pd
import numpy as np

#Set to my content

from google.colab import drive
drive.mount('/content/drive')
import os

#make sure to drag the checkpoint zip to the content file we are working from

Mounted at /content/drive


In [3]:
!pwd

/content


In [4]:
#Got the error below, but somehow the script after it works? haha

!unzip bleurt_20_D6_initial_checkpoint_test_8.1.zip

Archive:  bleurt_20_D6_initial_checkpoint_test_8.1.zip
   creating: my_new_bleurt_checkpoint/
  inflating: my_new_bleurt_checkpoint/events.out.tfevents.1690901735.d2a8ef82b746  
  inflating: my_new_bleurt_checkpoint/model.ckpt-0.data-00000-of-00001  
  inflating: my_new_bleurt_checkpoint/checkpoint  
  inflating: my_new_bleurt_checkpoint/model.ckpt-0.index  
  inflating: my_new_bleurt_checkpoint/model.ckpt-250.data-00000-of-00001  
   creating: my_new_bleurt_checkpoint/export/
   creating: my_new_bleurt_checkpoint/export/bleurt_best/
   creating: my_new_bleurt_checkpoint/export/bleurt_best/1690902769/
  inflating: my_new_bleurt_checkpoint/export/bleurt_best/1690902769/bert_config.json  
   creating: my_new_bleurt_checkpoint/export/bleurt_best/1690902769/variables/
  inflating: my_new_bleurt_checkpoint/export/bleurt_best/1690902769/variables/variables.index  
  inflating: my_new_bleurt_checkpoint/export/bleurt_best/1690902769/variables/variables.data-00000-of-00001  
  inflating: my_new

In [5]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip .
!unzip BLEURT-20-D6.zip

--2023-08-01 22:29:35--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 173.194.213.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171135344 (163M) [application/octet-stream]
Saving to: ‘BLEURT-20-D6.zip’

BLEURT-20-D6.zip    100%[===================>] 163.21M  30.4MB/s    in 5.4s    

2023-08-01 22:29:41 (30.4 MB/s) - ‘BLEURT-20-D6.zip’ saved [171135344/171135344]

--2023-08-01 22:29:41--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2023-08-01 22:29:41--
Total wall clock time: 5.6s
Downloaded: 1 files, 163M in 5.4s (30.4 MB/s)
Archive:  BLEURT-20-D6.zip
   creating: BLEURT-20-D6/
  inflating: BLEURT-20-D6/bert_config.json  
  inflating: BLEURT-20-D6/saved_model.pb  
   creating: BLEUR

In [6]:
!pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116
Receiving objects: 100% (134/134), 31.28 MiB | 32.09 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [7]:
!pip install ./bleurt

Processing ./bleurt
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456762 sha256=79fdcc8a10fe37cdf1ed7ade7746a4c48cf65777340aa995723c5872d3d21705
  Stored in directory: /tmp/pip-ephem-wheel-cache-_zpgwkb0/wheels/92/4f/fb/afa555fa27aa9e2c7958df797a62cc4e74f0f459cec9c4fa7c
Successfully built BLEURT


In [8]:
!pwd

/content


In [12]:
df_test = pd.read_csv("test_set.csv")

In [13]:
df_test = df_test.reset_index()

In [14]:
df_test.head()

,index,Unnamed: 0,lp,src,mt,ref,system,annotators,domain,year,score
0,0,16635,zh-en,在位于浙江海宁的正泰新能源光伏电池车间，最近质检员身旁多了位装载有阿里云人工智能的新“同事 ” 。,In the Zhengtai New Energy photovoltaic cell w...,In the CHNT New Energy Photovoltaic Cell Works...,DiDi_NLP.401,3,news,2020,-9.0
1,1,87398,en-de,After contacting two Apple Support specialists...,Nachdem ich mich an zwei Apple-Support-Experte...,"Ich kontaktierte Apple Support-Spezialisten, u...",bleurt_bestmbr,1,social,2022,-2.0
2,2,113386,zh-en,从结束文革定义真理标准，发展经济提高科技以及促进行政制度化和人权保障等方面，右派的积极意义不...,The positive roles played by the right wing sh...,The positive significance of the Rightists sho...,refB,1,social,2022,-0.0
3,3,119101,zh-en,文摄视觉美学摄影馆优选品质店 暂时还没有分店,Wenshu Visual Aesthetics Photo Studio Preferre...,Wenshe Visual Aesthetics Photography Studio Pr...,Online-Y,1,ecommerce,2022,-5.0
4,4,97729,en-de,I understand would you please check if the ebo...,"Ich verstehe, würdest du bitte überprüfen, ob ...","Ich verstehe, würden Sie bitte prüfen, ob das ...",Online-Y,1,conversation,2022,-2.0


In [15]:
#we have to turn out df_test to reference and candidate files
def create_bleurt_test_file(df, cand_filename = 'candidates', ref_filename = 'references'):
  os.system('rm ' + cand_filename)
  os.system('rm ' + ref_filename)
  os.system('touch ' + cand_filename)
  os.system('touch ' + ref_filename)
  cand_file = open(cand_filename, "a")
  ref_file = open(ref_filename, "a")

  for i in range(len(df['mt'])):
    cand_file.write(str(df['mt'][i]) + "\n")
    ref_file.write(str(df['ref'][i]) + "\n")

create_bleurt_test_file(df_test)

In [16]:
!mv bleurt/bleurt ./bleurt_temp
!mv bleurt bleurt_outer
!mv bleurt_temp bleurt

In [17]:
from bleurt import score
from my_new_bleurt_checkpoint import *

checkpoint = "/content/my_new_bleurt_checkpoint/export/bleurt_best/1690902769"
references = ["This is a test."]
candidates = ["This is the test."]

scorer = score.BleurtScorer(checkpoint)
scores = scorer.score(references=references, candidates=candidates)
assert isinstance(scores, list) and len(scores) == 1
print(scores)

[-0.8936223983764648]


In [18]:
#run the whole thing

df_subset = df_test
references = df_subset['ref']
candidates = df_subset['mt']

print(list(references), '\n', list(candidates))
scores = scorer.score(references = references, candidates = candidates)
print(scores)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[-3.101423978805542, -2.7893519401550293, -3.638044595718384, -1.5469579696655273, -2.17372727394104, -2.0512609481811523, -1.0599486827850342, -0.8344556093215942, -3.223202705383301, -1.9743616580963135, -0.8665311336517334, -2.18033766746521, -0.06542983651161194, 0.5564363598823547, -1.8764352798461914, -1.3716129064559937, -3.1521263122558594, -2.5916054248809814, -1.8694376945495605, 0.06973262131214142, -3.0001611709594727, -2.9358937740325928, -3.7493979930877686, -1.7459802627563477, -0.8919488191604614, -1.893864393234253, -3.6788651943206787, -1.4892964363098145, -1.4719079732894897, -2.2695462703704834, -1.342475175857544, -3.3834526538848877, -3.7155094146728516, -3.249929189682007, -3.6736092567443848, -2.3509938716888428, -1.638774037361145, -0.8168679475784302, -1.6132456064224243, -1.4802663326263428, -1.1232554912567139, -1.7277734279632568, -3.3150856494903564, -1.345693588256836, -2.7912585735321045, -2.1366114616394043, -2.4728567600250244, -3.6606509685516357, -3.

In [19]:
#add scores as new column

df_test['bluert20_D6_raw'] = scores

In [20]:
#expot to csv

df_test.to_csv('df_test_with_raw_scores_8_1.csv')